In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [1]:
!pip install espnet

In [2]:
!pip install espnet_model_zoo
!pip install jedi

  Using cached espnet_model_zoo-0.1.7-py3-none-any.whl.metadata (10 kB)
Using cached espnet_model_zoo-0.1.7-py3-none-any.whl (19 kB)
  Using cached jedi-0.19.2-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.2-py2.py3-none-any.whl (1.6 MB)


In [3]:
!pip install numpy==1.26

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 17.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
espnet 202506 requires numpy<1.24, but you have numpy 1.26.0 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.0 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.0 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.0 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", 

In [1]:
!apt-get install -y portaudio19-dev
!pip install pyaudio
!pip install ffmpeg-python
!pip install sounddevice
!pip install pyworld
!pip install librosa python_speech_features

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 0s (510 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 126380 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.

In [2]:
import sys
import espnet
import argparse
import numpy as np
import wave
import scipy
import pyworld

import string
import torch
import sounddevice as sd
from scipy.io.wavfile import write
import librosa
import soundfile as sf

from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text

Failed to import Flash Attention, using ESPnet default: No module named 'flash_attn'


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.
/usr/local/lib/python3.11/dist-packages/espnet2/enh/encoder/stft_encoder.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)
/usr/local/lib/python3.11/dist-packages/espnet2/enh/layers/uses2_swin.py:329: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)


In [3]:
# -------- RECORD AUDIO --------------

from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [27]:
import torch
import torchaudio
import librosa
import numpy as np
from scipy.interpolate import interp1d

def feature_extraction(npz_path):

    # 1. Load audio (ensure shape [1, T])
    waveform, sr = librosa.load('drive/My Drive/asr_files/recording/recording.wav', sr=16000)
    waveform = torch.from_numpy(waveform).float().unsqueeze(0)  # shape: [1, T]

    # 2. Compute FBANK (80-dim)
    # * PARAMETERS MIGHT REQUIRE TUNING BUT I DON'T KNOW EXACT PARAMS YET
    fbank_feats = torchaudio.compliance.kaldi.fbank(
        waveform,
        num_mel_bins=80,
        sample_frequency=sr,
        use_log_fbank=True,
        frame_length=25.0, # *
        frame_shift=10.0,  # *
    )  # shape: [frames, 80]

    # 3. Compute Pitch (3-dim) using librosa
    hop_length = 160  # 10ms at 16kHz *
    win_length = 400  # 25ms at 16kHz *
    pitch, voiced_flag, voiced_probs = librosa.pyin(
        waveform.squeeze().numpy(),  # input shape: [T]
        fmin=librosa.note_to_hz('C2'),
        fmax=librosa.note_to_hz('C7'),
        sr=sr,
        frame_length=win_length,
        hop_length=hop_length,
    )  # Each output shape: [T_orig]

    # Stack pitch features into [T_orig, 3]
    pitch_features = np.column_stack([
        np.nan_to_num(pitch),          # Replace NaN with 0
        np.nan_to_num(voiced_flag),
        np.nan_to_num(voiced_probs),
    ])

    '''
    # 4. Resample pitch features to match FBANK frames
    def resample_features(source, target_len):
        """Resamples source (T_orig, 3) to (target_len, 3) using linear interpolation."""
        source_len = source.shape[0]
        x_orig = np.linspace(0, 1, source_len)
        x_new = np.linspace(0, 1, target_len)
        interpolator = interp1d(x_orig, source, axis=0, kind='linear', fill_value="extrapolate")
        return interpolator(x_new)

    target_frames = fbank_feats.shape[0]
    pitch_resampled = resample_features(pitch_features, target_frames)

    # 5. Combine into 83-dim features
    fbank_pitch = np.hstack([fbank_feats.numpy(), pitch_resampled])
    '''

    # 4. Truncate to match the shorter length
    min_len = min(fbank_feats.shape[0], pitch_features.shape[0])
    fbank_feats = fbank_feats[:min_len]
    pitch_features = pitch_features[:min_len]

    # 5. Combine into 83-dim features
    fbank_pitch = np.hstack([fbank_feats.numpy(), pitch_features])

    print("Final shape:", fbank_pitch.shape)  # [frames, 83]

    # 6. Normalization
    stats      = np.load(npz_path)
    count      = stats["count"]
    sum_       = stats["sum"]
    sum_square = stats["sum_square"]

    mean = sum_ / count
    variance = (sum_square / count) - (mean ** 2)
    std = np.sqrt(np.maximum(variance, 1e-10))

    feats = ((fbank_pitch - mean) / std).astype(np.float32)

    return feats

In [36]:
def setup_asr_model():
    dir1             = "drive/My Drive/asr_files/asr_train_asr_transformer_fbank_pitch_my_bpe2500"
    asr_train_config = f"{dir1}/config.yaml"
    asr_model_file   = f"{dir1}/valid.acc.ave.pth"
    bpemodel_path    = "drive/My Drive/asr_files/bpe_unigram2500/bpe.model"
    # Load ASR model
    speech2text = Speech2Text(
        asr_train_config=asr_train_config,
        asr_model_file=asr_model_file,
        token_type='bpe',
        bpemodel=bpemodel_path,
        device="cpu",  # or "cuda" if you have GPU
        minlenratio=0.0,
        maxlenratio=0.0,
        ctc_weight=0.3,
        beam_size=20,
        batch_size=0,
        lm_weight=1.2,
        nbest=1,
    )
    return speech2text


def text_normalizer(text):
    text = text.upper()
    return text.translate(str.maketrans('', '', string.punctuation))

In [38]:
# ----- Record and STT --------------

if __name__ == "__main__":

    SAMPLING_RATE = 16000
    npz_path = "drive/My Drive/asr_files/asr_stats_fbank_pitch_my_bpe2500/train/feats_stats.npz"

    audio, sr = get_audio()

    print(audio.shape) # (samples,) means mono; (samples, 2) means stereo
    print(f'Sample Rate: {sr}Hz')

    scipy.io.wavfile.write('drive/My Drive/asr_files/recording/recording.wav', sr, audio)
    # audio, sr = librosa.load('drive/My Drive/asr_files/recording/recording.wav', sr=SAMPLING_RATE)
    # print(f'Resampled Rate: {sr}Hz')

    # feats   = extract_fbank_pitch(audio, sr, cmvn_file)
    feats     = feature_extraction(npz_path=npz_path)
    print(feats.shape)

    # Initialize model and transcribe
    speech2text = setup_asr_model()
    nbests      = speech2text(feats)
    text, *_    = nbests[0]

    print(f"\nTranscription:\n{text_normalizer(text)}")

(109440,)
Sample Rate: 48000Hz
Final shape: (226, 83)
(226, 83)


/usr/local/lib/python3.11/dist-packages/espnet2/asr/espnet_model.py:402: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(self.autocast_frontend, dtype=autocast_type):



Transcription:
တိ ရစ္ဆာန် အ စာ ကို ဘယ် လို အ ကြံ ပြု နိုင် လဲ


In [39]:
# ------ Use Existing .wav File -----------

if __name__ == "__main__":

    npz_path = "drive/My Drive/asr_files/asr_stats_fbank_pitch_my_bpe2500/train/feats_stats.npz"

    feats     = feature_extraction(npz_path=npz_path)
    print(feats.shape)

    # Initialize model and transcribe
    speech2text = setup_asr_model()
    nbests      = speech2text(feats)
    text, *_    = nbests[0]

    print(f"\nTranscription:\n{text_normalizer(text)}")

Final shape: (226, 83)
(226, 83)

Transcription:
တိ ရစ္ဆာန် အ စာ ကို ဘယ် လို အ ကြံ ပြု နိုင် လဲ
